**Preparation of SEC Complaint Documents, Enron Annual Reports, and Enron Case Study for RAG**





Using Unstructured.io for quick ingestion of the annual report documents. Not as precise as rule-based parsing done in Milestone 1, but quick and easy.


In [3]:
pip install -q unstructured


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
import os
import pickle

from collections import defaultdict

import pandas as pd

from unstructured.partition.text import partition_text


Process Enron Good Bad Lessons Case Study (which we formatted as a text document in pipe delimited format where section titles are GroupName equivalents)

In [19]:
file_path = "/work/data/Enron_Good_Bad.txt"

# Specify header=None to avoid assuming the first line is a header
df = pd.read_csv(
    file_path,
    delimiter="|",
    header=None,
    names=["GroupName", "grouped_text"],
    skip_blank_lines=False,
    encoding="utf-8",
    engine="python",
)

df["GroupName"] = df["GroupName"]
df["grouped_text"] = df["grouped_text"]

print(df)

                                           GroupName  \
0                                     Enron Overview   
1  Why didn’t Arthur Andersen provide adequate as...   
2  Could the SEC and other regulators have done m...   
3  Why didn’t external financial analysts detect ...   
4  Why was Sherron Watkins a lone voice in asking...   
5  What changes have occurred or are expected to ...   
6                       Enron Timeline: 1988 to 1997   
7           The Meteoric Rise Of Enron: 1998 to 2001   
8                  A Sudden Collapse July 2001 to ??   

                                        grouped_text  
0  Enron created 3500–4000 Special Purpose Entiti...  
1  Critics note that Arthur Anderson (AA) was too...  
2  In 2000, half of Arthur Andersen’s $52 million...  
3  How can we detect other companies in similar d...  
4  As is common in a firm with a powerful, domine...  
5  The Enron scandal itself has already brought s...  
6  In 1985, Enron started as Houston Natural Gas....  

In [20]:
# Path to output pickle file
pickle_path = "/work/data/EnronGoodBad.pkl"

# Save the dataframe to a pickle so we have this intermediate source in expected column format
df.to_pickle(pickle_path)

print(f"Dataframe saved to pickle file: {pickle_path}")

Dataframe saved to pickle file: /work/data/EnronGoodBad.pkl


Ingestion loop for Annual Report (10-K text files) follows. GroupNames created by the Title elements output with Unstructured.io.

(Not an exact process - but part of the experiment regarding quick ingestion.)

In [21]:
# Used by functions below
def parse_elements_to_dataframe(elements):
    """
    This function parses elements based on category and builds a DataFrame with grouped text information.
    """
    current_group_name = None
    current_group_text = ""
    # Create an empty list to store DataFrames
    df_list = []

    for element in elements:
        if element.category == "Title":
            # Start a new group with the Title text
            current_group_name = element.text
            current_group_text = ""
        elif element.category == "NarrativeText":
            # Append NarrativeText to the current group, adding a space separator if needed
            current_group_text += (" " if current_group_text else "") + element.text
        else:
            # Ignore elements from other categories
            pass

        # If a group name is available and there's content, create a new DataFrame and append
        if current_group_name and current_group_text:
            df_list.append(
                pd.DataFrame(
                    {
                        "GroupName": [current_group_name],
                        "grouped_text": [current_group_text],
                    }
                )
            )
            # Reset current variables for next group
            current_group_name = None
            current_group_text = ""

    # Concatenate all DataFrames in list into single DataFrame
    df = pd.concat(df_list, ignore_index=True)
    return df

In [22]:
def process_fin_documents(document_paths, output_dir=None):
    """
    This function iterates through document paths, parses each document, and saves the resulting DataFrame as a CSV file.
    Args:
        document_paths (list): list of document paths.
        output_dir (str, optional): directory to save the output CSV files. If not provided,
            files are saved in the same directory as the documents. Defaults to None.
    """
    for document_path in document_paths:
        # Extract filename without path and extension
        filename = os.path.splitext(os.path.basename(document_path))[0]

        # Determine output filename based on output_dir
        if output_dir:
            output_file = os.path.join(output_dir, f"{filename}_df_group.csv")
        else:
            output_file = f"{filename}_df_group.csv"

        # Partition the document and get the DataFrame
        elements = partition_text(filename=document_path)
        parsed_df = parse_elements_to_dataframe(elements)

        # Save the DataFrame to a CSV file
        parsed_df.to_csv(output_file, index=False)

        print(f"Parsed document: {document_path}, saved as: {output_file}")

In [23]:
# Your list of document paths

document_paths = [
    "/work/data/2000_10k.txt",
    "/work/data/1999_10k.txt",
    "/work/data/1998_10k.txt",
    "/work/data/1997_10k.txt",
    "/work/data/1996_10k.txt",
    "/work/data/1995_10k.txt",
    "/work/data/1994_10k.txt",
    "/work/data/1993_10k.txt",
]

# Save to output directory
output_dir = "/work/data/"  # Desired output directory
process_fin_documents(document_paths, output_dir)

Parsed document: /work/data/2000_10k.txt, saved as: /work/data/2000_10k_df_group.csv
Parsed document: /work/data/1999_10k.txt, saved as: /work/data/1999_10k_df_group.csv
Parsed document: /work/data/1998_10k.txt, saved as: /work/data/1998_10k_df_group.csv
Parsed document: /work/data/1997_10k.txt, saved as: /work/data/1997_10k_df_group.csv
Parsed document: /work/data/1996_10k.txt, saved as: /work/data/1996_10k_df_group.csv
Parsed document: /work/data/1995_10k.txt, saved as: /work/data/1995_10k_df_group.csv
Parsed document: /work/data/1994_10k.txt, saved as: /work/data/1994_10k_df_group.csv
Parsed document: /work/data/1993_10k.txt, saved as: /work/data/1993_10k_df_group.csv


In [24]:
def load_and_process_fin_documents(document_paths):
    """
    This function filters the 10K csvs to remove sections that don't appear useful and concats them to a new DataFrame.
    """

    dfs = []

    for path in document_paths:

        df = pd.read_csv(path)

        # Define GroupName starting words for exclusion

        words_to_exclude = [
            "CONSENT OF INDEPENDENT PUBLIC ACCOUNTANTS",
            "POWER OF ATTORNEY",
        ]

        # Convert df to all strings since partition seems to have added mixed types

        df["GroupName"] = df["GroupName"].astype(str)

        df["grouped_text"] = df["grouped_text"].astype(str)

        # Use the .str.startswith() method with the OR operator to create a boolean mask

        mask = df["GroupName"].apply(
            lambda x: not any(x.startswith(word) for word in words_to_exclude)
        )

        # Apply mask to filter DataFrame

        df_filtered = df.loc[mask]

        # Extract first four characters of the filename (excluding path)

        filename = os.path.basename(path).split(".")[0][:4]

        # Create string to prepend to GroupName

        prepend_str = f"{filename} Annual Report: "

        # Add prepend string to GroupName column

        df_filtered.loc[:, "GroupName"] = prepend_str + df_filtered["GroupName"]

        # Append to list of dataframes

        dfs.append(df_filtered)

    # Concat all dataframes

    fin_combined_df = pd.concat(dfs, ignore_index=True)

    return fin_combined_df

In [25]:
document_paths = [
    "/work/data/2000_10k_df_group.csv",
    "/work/data/1999_10k_df_group.csv",
    "/work/data/1998_10k_df_group.csv",
    "/work/data/1997_10k_df_group.csv",
    "/work/data/1996_10k_df_group.csv",
    "/work/data/1995_10k_df_group.csv",
    "/work/data/1994_10k_df_group.csv",
    "/work/data/1993_10k_df_group.csv",
]
fin_combined_df = load_and_process_fin_documents(document_paths)
print(fin_combined_df.head())
print(fin_combined_df.tail())

                                           GroupName  \
0       2000 Annual Report: Commission File Number 1   
1                        2000 Annual Report: GENERAL   
2              2000 Annual Report: BUSINESS SEGMENTS   
3  2000 Annual Report: TRANSPORTATION AND DISTRIB...   
4  2000 Annual Report: Interstate Transmission of...   

                                        grouped_text  
0  ENRON CORP. (Exact name of registrant as speci...  
1  Headquartered in Houston, Texas, Enron Corp., ...  
2  Enron has divided its operations into the foll...  
3  Enron's Transportation and Distribution busine...  
4  Enron and its subsidiaries operate domestic in...  
                                        GroupName  \
842        1993 Annual Report: Date: July 1, 1993   
843                1993 Annual Report: Exhibit 12   
844  1993 Annual Report: ENRON CORP. SUBSIDIARIES   
845                1993 Annual Report: Gentlemen:   
846                1993 Annual Report: Gentlemen:   

              

In [26]:
fin_combined_df.to_csv("/work/data/fin_combined_df.csv", index=False)

Now the SEC Complaint document ingestion, using the rules-based extraction and parsing of the data from Milestone 1.

In [27]:
def load_and_process_case_documents(document_paths):
    """
    This function filters the SEC case doc csvs to remove sections that don't appear useful and concats them to a new DataFrame.
    """

    dfs = []

    for path in document_paths:

        df = pd.read_csv(path)

        # Define GroupName starting words for exclusion

        words_to_exclude = ["Violation", "Violations", "Aiding"]

        # Use .str.startswith() method with OR operator to create boolean mask

        mask = df["GroupName"].apply(
            lambda x: not any(x.startswith(word) for word in words_to_exclude)
        )

        # Apply mask to filter

        df_filtered = df[mask]

        # Append to list of dataframes

        dfs.append(df_filtered)

    # Concat all dataframes

    case_combined_df = pd.concat(dfs, ignore_index=True)

    return case_combined_df

In [28]:
document_paths = [
    "/work/data/18776_df_group.csv",
    "/work/data/20058_df_group.csv",
    "/work/data/20441_df_group.csv",
    "/work/data/18435_df_group.csv",
]
case_combined_df = load_and_process_case_documents(document_paths)
print(case_combined_df.head())
print(case_combined_df.tail())

                                           GroupName  \
0  The Objectives And Roots Of The Scheme To Defraud   
1  Use of Special Purpose Entities and LJM Partne...   
2                        Creation of LJM Partnership   
3                                    "Raptor" Hedges   
4  Manufacturing Earnings and Concealing Debt thr...   

                                        grouped_text  
0  The objectives of the scheme to defraud carrie...  
1  As part of the scheme to defraud, Skilling, Ca...  
2  In June 1999, Skilling, Causey, and others sou...  
3  Beginning in the spring of 2000, Enron and LJM...  
4  In addition to the fraudulent Raptor hedging d...  
                                            GroupName  \
65  Concealment of Uncollectible Receivables Owed ...   
66  Concealment of EES Failures by Manipulating Re...   
67          Fraudulent Valuation of "Merchant" Assets   
68  Other Manipulative Devices Used in Enron Whole...   
69                         Delainey's Insider Tr

In [29]:
case_combined_df.to_csv("/work/data/case_combined_df.csv", index=False)

Now concat the dataframes.

In [30]:
def load_and_concatenate_dfs(path1, path2, path3, output_path):
    """
    Loads dataframes from CSV and pickle files, concatenates them, and saves the result.

    Args:
        path1 (str): Path to the first CSV file.
        path2 (str): Path to the second CSV file.
        path3 (str): Path to the pickle file.
        output_path (str): Path to save the concatenated dataframe.

    Returns:
        pandas.DataFrame: The concatenated dataframe.
    """

    # Load CSV dataframes (with explicit UTF-8 encoding)
    df1 = pd.read_csv(path1, encoding="utf-8")
    df2 = pd.read_csv(path2, encoding="utf-8")

    # Load pickle dataframe
    with open(path3, "rb") as f:
        df3 = pd.read_pickle(f)  # Use pd.read_pickle() for pickle files

    # Ensure all column values are strings
    df1 = df1.astype(str)
    df2 = df2.astype(str)
    df3 = df3.astype(str)  # If necessary, convert df3 columns as well

    # Concat dataframes vertically
    combined_df = pd.concat([df1, df2, df3], ignore_index=True)

    # Save concatenated dataframe as CSV and pickle
    combined_df.to_csv(output_path, index=False)

    # with open('/work/data/final_combined.pkl', 'wb') as f:
    with open(output_path, "wb") as f:
        pickle.dump(combined_df, f)

    return combined_df

In [31]:
# Define the paths
path1 = "/work/data/case_combined_df.csv"
path2 = "/work/data/fin_combined_df.csv"
path3 = "/work/data/EnronGoodBad.pkl"
output_path = "/work/data/final_combined_df.csv"

# Execute function
final_df = load_and_concatenate_dfs(path1, path2, path3, output_path)
print(f"Dataframe saved to {output_path}")


# Save concatenated dataframe to specified output path as pickle
with open("/work/data/final_combined_df.pkl", "wb") as f:
    pickle.dump(final_df, f)

Dataframe saved to /work/data/final_combined_df.csv


final_combined_df.pkl will be used in the next RAG notebook for the augmented retrieval.